In [ ]:
#SAC_3 custom env healthy reward 7 ,rewards = healthy_reward - forward_reward

In [1]:
import sys
sys.path.insert(0, "/home/vardan/anaconda3/envs/pytorch/projects/git_humanoid/humanoid/GYM_git")


In [2]:
import gymnasium
print(gymnasium.__file__)


/home/vardan/anaconda3/envs/pytorch/projects/git_humanoid/humanoid/GYM_git/gymnasium/__init__.py


In [3]:
from gymnasium.envs.registration import register

register(
    id="HumanoidV5Custom-v0",
    entry_point="gymnasium.envs.custom_envs.humanoid_v5_custom:HumanoidEnv",
    max_episode_steps=1000,
)


In [4]:
import gymnasium.envs.custom_envs.humanoid_v5_custom  # This triggers the register()
env = gymnasium.make("HumanoidV5Custom-v0", render_mode=None)


In [5]:
obs, _ = env.reset()
done = False

while not done:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print(reward)
    done = terminated or truncated

env.close()


6.935595710143438
6.875816164028531
6.903439662175464
6.908488514643672
6.861877617786858
6.90730357797555
6.909159567628249
6.908698913668964
6.861620117764319
4.125507380102762
6.472754566912852
6.66363123786341
6.2931817939277055
6.408277516722705
6.287960679537309
6.443458838425887
6.527960652519183
6.505836109634461
6.470925397409035
6.507040879687444
6.473642474693984
6.478821652116224
-0.5624433214913045


In [6]:
from stable_baselines3 import SAC, TD3, A2C
import os
import argparse
from stable_baselines3.common.noise import NormalActionNoise
import numpy as np

2025-05-03 22:27:22.602331: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:


# Create base directories
model_base_dir = "models"
log_base_dir = "logs"
os.makedirs(model_base_dir, exist_ok=True)
os.makedirs(log_base_dir, exist_ok=True)

name = "SAC_"

# Find next available index
existing_dirs = [d for d in os.listdir(log_base_dir) if d.startswith(name) and d[4:].isdigit()]
next_index = max([int(d[4:]) for d in existing_dirs], default=-1) + 1

# Create log and model directories with same name
log_dir = os.path.join(log_base_dir, f"{name}{next_index}")
model_dir = os.path.join(model_base_dir, f"{name}{next_index}")
os.makedirs(log_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

def train(model, timesteps):
    iters = 0
    while True:
        iters += 1
        model.learn(total_timesteps=timesteps, reset_num_timesteps=False)
        # Save to models/SAC_X/
        model.save(f"{model_dir}/model_{model.num_timesteps}")
    return model
# def train(model, timesteps, ):

    

#     iters = 0
#     while True:
#         iters += 1
#         model.learn(total_timesteps=timesteps, reset_num_timesteps=False)
#         model.save(f"{model_dir}{name}{next_index}{model.num_timesteps - model.num_timesteps % 1000}")
#     return model





In [8]:
# action_noise = NormalActionNoise(mean=np.zeros(env.action_space.shape), 
#                                  sigma=0.15 * np.ones(env.action_space.shape))
# # model = SAC('MlpPolicy', env, verbose=1, device='cuda', tensorboard_log=log_dir)
# model = SAC(
#     "MlpPolicy",
#     env,
#     verbose=1,
#     device="cuda",
#     tensorboard_log=log_dir,
#     learning_rate=0.0004,        # Default, but can be reduced for more exploration
#     batch_size=512,              # Increase batch size for stability
#     tau=0.004,                   # Lower tau to slow down target updates
#     gamma=0.98,                  # Slightly reduce gamma to value short-term rewards more
#     ent_coef=0.06,               # Encourage more entropy (more randomness)
#     target_update_interval=2,     # Update targets slightly less frequently
#     action_noise=action_noise,     # Inject additional exploration noise
# )

In [9]:
model = SAC.load("models/SAC_2/SACv1_962000", verbose=1, device='cuda', tensorboard_log=log_dir)
model.set_env(env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model = train(model, 20000)

Logging to logs/SAC_3/SAC_0


/home/vardan/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  return F.linear(input, self.weight, self.bias)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 926      |
|    ep_rew_mean     | 5.72e+03 |
| time/              |          |
|    episodes        | 2792     |
|    fps             | 35       |
|    time_elapsed    | 13       |
|    total_timesteps | 962845   |
| train/             |          |
|    actor_loss      | -266     |
|    critic_loss     | 4.78     |
|    ent_coef        | 0.05     |
|    learning_rate   | 0.0003   |
|    n_updates       | 962743   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 898      |
|    ep_rew_mean     | 5.54e+03 |
| time/              |          |
|    episodes        | 2796     |
|    fps             | 36       |
|    time_elapsed    | 32       |
|    total_timesteps | 963564   |
| train/             |          |
|    actor_loss      | -254     |
|    critic_loss     | 3.87     |
|    ent_coef        | 0.05     |
|    learning_